# 10:50 AM: Requests under the hood

[Cory Benfield](https://lukasa.co.uk)

[This is the requests library](https://github.com/kennethreitz/requests)

Why does code get messy?
* "You end up with a function where the only defined behavior is ... well it does whatever it does now, so let's not mess with it"
* "When it works, we don't think about it"
* Pragmatism
* The quick hacky fix is never temporary.
* Users don't complain about it
* Over time you enshrine every edge case ever into your code.
* Things are complex AND they evolve


**Hero-worship, while fine, is fundamentally predicated on a lie. Our heros are not dramatically better at writing better code than us**

* Any non-trivial software is flawed.
* We need to not be afraid of our mistakes.

Questions/Discussions:
* How do you balance the need to do things right with a need to immediately fix things? At some point, the hacks are okay, but acknowledge that they are hacks.
* Is it possible to deossify a codebase? When the combined weight of all the historical decisions make it impossible to keep up, someone else will probably come up with something better.

# 11:30 AM: Type uWSGI; Pres Enter; What Happens?

[Asheesh Laroia](https://github.com/paulproteus); [Philip James](https://twitter.com/phildini)

Previous talk: Type Python pres enter, what happens


[Youtube link](https://www.youtube.com/watch?v=XVhSjZYwZJo)

[Slides](https://speakerdeck.com/phildini/type-python-press-enter-what-happens-pydx-2015)

Running UWSGI

```bash 
uwsgi --master --http :8000 --module catserve.wsgi 
```

**How does uWSGI handle processes?**

* WSGI is a synchronous protocol

To un-ruin the service, run two worker processes, and there are now 2 uWSGI workers.
```bash 
uwsgi --master --http :8000 --module catserve.wsgi -p 2
```

**How does uWSGI handle networking?**

* Userland communicates with the kernel through syscalls

To connect to the uWSGI server?
1) uWSGI needs to issue the socket syscall to issue the socket that will connect to the port on the kernel
    * uWSGI is getting back a way of handling the socket
2) Bind connects the port to the socket
3) Listen() tells the kernel to expose the port and make it available for listening
4) This is when uWSGI spawns its workers. Keeps initial file descriptors. These worker processes issue epoll_wait() which tell the kernel that they are ready and waiting for any connections.

So...when the browser opens the port:
* Both processes try to "pick up the phone"
* One wins, and handles the connection through a separate socket.

**Why use uWSGI?**
* Graceful code reloading
* Tuneability
* Security
* Making config files
* Other features

How does code reloading work?

* With a new version of the code
* uWSGI sends sighup to the worker processes
* uWSGI tells anything not working to stop, and marks another one that it "needs to exit" after
* Always checks if it should exit before calling the epoll_wait()
* uWSGI erases its memory and creates a new process with the same pid and same socket, which never closes: thus can queue new code.
* Always waits for all worker processes to exit, and by default waits 60 seconds. You can make it reload worker processes one by one.

**Making sure the resource usage meets your needs**
* The `-p` parameter lets you change the # of processes

**Security**

> "Security is a word, that when you say it, everyone in the room feels inadequate"

```
GET / HTTP/1.1
HOST: catserve.io
```
vs. 
```
GET / HTTP/1.1
HOST: catserve.io
HOST: catserve.biz
```

* Interactions between the thing running your code and the rest of the stuff it runs could turn into security issues

** Config Files **

Translate the bash command from the beginning int:
```
[uwsgi]
master = 1
http = 8000
module = catserve.wsgi
processes = 2
```

**Features**
* Static file serving (high performance implementation in C)
* Max requests per worker
* Queuing systems
* HTTPS support, HTTP2 support
* uwsgitop
* memory-report
* async